LCEL -> Built-In Functions

1. Runnable PassThrough (Pass through - no changes in that - pass same thing without changing)
2. Runnable Parallel (Multiple seq of chain)
3. Runnable Lambda (Custom Function)

1. Runnable PassThrough


In [2]:
import os
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage

llm_model = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-70b-8192"
)

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()
chain.invoke('prince')

'prince'

2. Runnable Lambda

In [5]:
from langchain_core.runnables import RunnableLambda

In [6]:
def russian_lastname(name : str) -> str:
    return f"{name}ovich"

In [7]:
chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [11]:
chain.invoke("MahimaShetty")

'MahimaShettyovich'

In [ ]:
def russian_lastname_dict(input_dict: dict) -> str:
    name = input_dict["name"]
    return f"{name}ovich"

In [13]:
chain = RunnablePassthrough() | RunnableLambda(russian_lastname_dict)

In [14]:
chain.invoke({"name":"MahimaShetty"})

'MahimaShettyovich'

3. Runnable Parallel

In [18]:
from langchain_core.runnables import RunnableParallel
from langchain.prompts import ChatPromptTemplate
chain = RunnableParallel(
    {"operation_a" : RunnablePassthrough(),
    "operation_b" : RunnableLambda(russian_lastname)}
)

In [19]:
chain.invoke("Mahima")

{'operation_a': 'Mahima', 'operation_b': 'Mahimaovich'}

In [20]:
prompt = ChatPromptTemplate.from_template(
    ("tell me a curious fact about {soccer_player}")
)


In [21]:
from langchain_core.output_parsers import StrOutputParser

In [25]:
chain = RunnableParallel(
    {"operation_a" : RunnablePassthrough(),
     "soccer_player" : RunnableLambda(russian_lastname_dict),
     "operation_c": RunnablePassthrough()}
) | prompt | llm_model | StrOutputParser()

In [26]:
chain.invoke({"name":"MahimaShetty"})

"I apologize, but I couldn't find any information on a person named MahimaShettyovich. It's possible that this person is not a public figure or doesn't have a significant online presence. Can I assist you with something else?"

Retrieval Chains

In [29]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings


In [30]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # Small, fast, reliable
)


C:\Users\msshe\AppData\Local\Temp\ipykernel_13916\687389532.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
c:\Users\msshe\Documents\Projects\Demo-Langchain\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
vectorstore = FAISS.from_texts(["learning logic focus on providing Data science, AI, DL, CV, Python"], embedding = embedding_model)

In [32]:
retriever = vectorstore.as_retriever()

In [33]:
template = """ Answer the question based only on the following content {context}   Question : {question}"""

In [34]:
prompt = ChatPromptTemplate.from_template(template)


In [35]:
retrieval_chain = RunnableParallel({"context": retriever, "question": RunnablePassthrough()}) | prompt | llm_model | StrOutputParser()

In [37]:
retrieval_chain.invoke("where did harrison work?")

'There is no information about Harrison or where he worked in the provided content. The content only mentions "learning logic" and lists some topics like Data Science, AI, DL, CV, and Python, but it does not mention Harrison or any person\'s workplace.'